In [1]:
import os
import glob
import json
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List, Tuple

## Funções Auxiliares (Carregamento, Salvamento e Visualização)

In [ ]:
def desserializar_keypoints(arr: np.ndarray) -> List[cv2.KeyPoint]:
    """
    Reconstrói lista de cv2.KeyPoint a partir do array (N,7).
    """
    kps = []
    for row in arr:
        k = cv2.KeyPoint(
            x=float(row[0]), y=float(row[1]), size=float(row[2]),
            angle=float(row[3]), response=float(row[4]),
            octave=int(row[5]), class_id=int(row[6])
        )
        kps.append(k)
    return kps

def carregar_features_npz(caminho_npz: str) -> Tuple[List[cv2.KeyPoint], np.ndarray, str]:
    """
    Lê um .npz salvo pela Etapa 2 e retorna (keypoints, descriptors, imagem_absoluta).
    """
    with np.load(caminho_npz, allow_pickle=True) as data:
        kps = desserializar_keypoints(data["keypoints"])
        desc = data["descriptors"]
        img_path = str(data["imagem_absoluta"])
    return kps, desc, img_path

def carregar_matches_npz(caminho_npz) -> List[cv2.DMatch]:
    """
    Carrega os matches salvos em .npz e restaura cada linha como um objeto cv2.DMatch.
    Retorna uma lista de cv2.DMatch.
    """

    with np.load(caminho_npz, allow_pickle=True) as data:
        arr = data["matches"]  # array shape (N, 3)
        matches = []
        for row in arr:
            # cv2.DMatch(queryIdx, trainIdx, distance)
            m = cv2.DMatch(_queryIdx=int(row[0]), _trainIdx=int(row[1]), _distance=float(row[2]))
            matches.append(m)
    return matches

## Pipeline Principal da Etapa 4

In [ ]:
def analisar_homogr_alinhamento(caminho_base_etapa2: str, caminho_base_etapa3: str, detectores: List[str], matchers: List[str]):
    """
    Pipeline da Etapa 4: Carrega features e matches, calcula homografia com RANSAC e alinha imagens com warpPerspective.
    """
    print(f"\n{'='*65}\nHomografia e alinhamento para o conjunto: '{os.path.basename(caminho_base_etapa3)}'\n{'='*65}")

    # Carrega a ordem das imagens
    caminho_json = os.path.join(caminho_base_etapa2, "ordem_imagens.json")
    try:
        with open(caminho_json, "r") as f:
            arquivos_de_imagem_abs = json.load(f)
    except FileNotFoundError:
        print(f"[Erro] 'ordem_imagens.json' não encontrado em '{caminho_base_etapa2}'. Abortando.")
        return
    
    # Itera sobre os detectores (sift, orb, etc.)
    for nome_detector in detectores:
        print(f"\n--- Detector: {nome_detector.upper()} ---")
        caminho_detector = os.path.join(caminho_base_etapa2, nome_detector)
        arquivos_feature_npz = sorted(glob.glob(os.path.join(caminho_detector, "*_features.npz")))
        if len(arquivos_feature_npz) < 2:
            continue

        # Itera sobre os matchers (bf, flann)
        for nome_matcher in matchers:
            print(f"  - Matcher: {nome_matcher.upper()}")
            caminho_matcher = os.path.join(caminho_base_etapa3, nome_detector, nome_matcher)
            arquivos_match_npz = sorted(glob.glob(os.path.join(caminho_matcher, "matches_*.npz")))

            # Cria o diretório de saída para esta combinação
            dir_saida = os.path.join("resultados_etapa4", os.path.basename(caminho_base_etapa3), nome_detector, nome_matcher)
            os.makedirs(dir_saida, exist_ok=True)

            # Itera sobre os pares de imagens
            # Carrega as features da Etapa 2 e matches da Etapa 3
            for i in range(len(arquivos_feature_npz) - 1):
                kps1, desc1, path1 = carregar_features_npz(arquivos_feature_npz[i])
                kps2, desc2, path2 = carregar_features_npz(arquivos_feature_npz[i+1])

                match_data = carregar_matches_npz(os.path.join(caminho_matcher, f"matches_{i:03d}_{i+1:03d}.npz"))

                if not kps1 or not kps2 or not match_data:
                    print("[Erro] Não foi possível carregar as features ou matches.")
                    return

                homografia = calcular_homogr(features, matches)

                if homografia is None:
                    print("[Erro] Não foi possível calcular a homografia.")
                    return

                imagens_alinhadas = alinhar_imgs(caminho_base_etapa3, homografia)

                if not imagens_alinhadas:
                    print("[Erro] Não foi possível alinhar as imagens.")
                    return
